In [2]:
file_path = "./data/csic_database.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(file_path)

print(df.head())

  Unnamed: 0 Method                                         User-Agent  \
0     Normal    GET  Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...   
1     Normal    GET  Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...   
2     Normal   POST  Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...   
3     Normal    GET  Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...   
4     Normal   POST  Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...   

     Pragma Cache-Control                                             Accept  \
0  no-cache      no-cache  text/xml,application/xml,application/xhtml+xml...   
1  no-cache      no-cache  text/xml,application/xml,application/xhtml+xml...   
2  no-cache      no-cache  text/xml,application/xml,application/xhtml+xml...   
3  no-cache      no-cache  text/xml,application/xml,application/xhtml+xml...   
4  no-cache      no-cache  text/xml,application/xml,application/xhtml+xml...   

                    Accept-encoding               Accept-charset language 

In [5]:
# example content

for index, row in df[:10].iterrows():
    print(row["content"])

nan
nan
id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito
nan
modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar
nan
id=2
nan
nan
errorMsg=Credenciales+incorrectas


In [9]:
# example urls

for index, row in df[:10].iterrows():
    print(row["URL"])


http://localhost:8080/tienda1/index.jsp HTTP/1.1
http://localhost:8080/tienda1/publico/anadir.jsp?id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito HTTP/1.1
http://localhost:8080/tienda1/publico/anadir.jsp HTTP/1.1
http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar HTTP/1.1
http://localhost:8080/tienda1/publico/autenticar.jsp HTTP/1.1
http://localhost:8080/tienda1/publico/caracteristicas.jsp?id=2 HTTP/1.1
http://localhost:8080/tienda1/publico/caracteristicas.jsp HTTP/1.1
http://localhost:8080/tienda1/publico/carrito.jsp HTTP/1.1
http://localhost:8080/tienda1/publico/entrar.jsp?errorMsg=Credenciales+incorrectas HTTP/1.1
http://localhost:8080/tienda1/publico/entrar.jsp HTTP/1.1


In [10]:

# remove http://localhost:8080/ from url
import re

# Pattern to match 'http://localhost:' followed by any port number
pattern = r'http://localhost:\d+'

# Function to clean URLs with type hints
def clean_url(url: str) -> str:
    return re.sub(pattern, '', url)


In [11]:
df['URL'] = df['URL'].apply(clean_url)

In [12]:
for index, row in df[:10].iterrows():
    print(row["URL"])

/tienda1/index.jsp HTTP/1.1
/tienda1/publico/anadir.jsp?id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito HTTP/1.1
/tienda1/publico/anadir.jsp HTTP/1.1
/tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar HTTP/1.1
/tienda1/publico/autenticar.jsp HTTP/1.1
/tienda1/publico/caracteristicas.jsp?id=2 HTTP/1.1
/tienda1/publico/caracteristicas.jsp HTTP/1.1
/tienda1/publico/carrito.jsp HTTP/1.1
/tienda1/publico/entrar.jsp?errorMsg=Credenciales+incorrectas HTTP/1.1
/tienda1/publico/entrar.jsp HTTP/1.1


In [13]:
def row_to_llm_input(row: pd.Series) -> str:
    return f"Method: {row['Method']}\nURL: {row['URL']}\nContent: {row['content']}"

df['llm_input'] = df.apply(row_to_llm_input, axis=1)

for index, row in df[:10].iterrows():
    print(row["llm_input"])

Method: GET
URL: /tienda1/index.jsp HTTP/1.1
Content: nan
Method: GET
URL: /tienda1/publico/anadir.jsp?id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito HTTP/1.1
Content: nan
Method: POST
URL: /tienda1/publico/anadir.jsp HTTP/1.1
Content: id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito
Method: GET
URL: /tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar HTTP/1.1
Content: nan
Method: POST
URL: /tienda1/publico/autenticar.jsp HTTP/1.1
Content: modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar
Method: GET
URL: /tienda1/publico/caracteristicas.jsp?id=2 HTTP/1.1
Content: nan
Method: POST
URL: /tienda1/publico/caracteristicas.jsp HTTP/1.1
Content: id=2
Method: GET
URL: /tienda1/publico/carrito.jsp HTTP/1.1
Content: nan
Method: GET
URL: /tienda1/publico/entrar.jsp?errorMsg=Credenciales+incorrectas HTTP/1.1
Content: nan
Method: POST
URL: /tienda1/publico/entrar.jsp HTTP/1.1
Content: errorMsg=Crede

In [15]:
# max length of llm_input

print(len(df["llm_input"].iloc[0]))

57


In [20]:
def row_to_label(row: pd.Series) -> str:
    # Create binary label: 0 for Normal, 1 for Anomalous
    label = 0 if row['Unnamed: 0'] == 'Normal' else 1
    return label


df['label'] = df.apply(row_to_label, axis=1)

In [21]:
df["label"].value_counts()

label
0    36000
1    25065
Name: count, dtype: int64

In [23]:
# Save new dataset
df.to_csv("./data/csic_database_labeled.csv", index=False)
